In [1]:
import os
home = os.path.expanduser('~')
tar_archive_path = os.path.join(os.curdir, 'raw_data',
                                '33m-russian-courts-cases-by-suvorov', 
                                'arb_sud.tar'
#                                 'project2.tar'
                               )

arb_sud_folder = os.path.join(
    os.curdir,
    'raw_data',
    '33m-russian-courts-cases-by-suvorov',
    'arb_sud'
)

import tarfile
tar_archive = tarfile.open(tar_archive_path, 'r')
import pandas as pd
import glob
import zlib
import gzip
from bs4 import BeautifulSoup
import json
from tqdm.notebook import tqdm
from IPython import display
import utils
import hashlib
import numpy as np
import random
import shutil
from importlib import reload
n_files = 14810696  # number of files in arb_sud.tar archive
reload(utils);

### <span style="font-family:Papyrus; font-size:1.5em;">This is an auxiliray notebook for data format manipulations </span>

In [469]:
case_to_category_dict = utils.get_case_to_category_mapping()
# pd.read_csv('разметка/2007_match_first.csv').head(2)

In [22]:
def read_document(archive, doc_link):
        doc_bin_gz =  archive.extractfile(doc_link).read()
        doc_bin_xml = zlib.decompress(doc_bin_gz, 16+zlib.MAX_WBITS)
        soup = BeautifulSoup(doc_bin_xml, 'lxml')
        return soup

---

# Reading torrent data

In [7]:
# solutions_info = json.load(open('solutions_info.json', 'r'))
# progress_counter = json.load(open('progress_counter.json', 'r'))[0]
# regions_parsed = set(json.load(open('regions_parsed.json', 'r')))

In [9]:
# pbar = tqdm(total=n_files)
# pbar.update(progress_counter)
# for region_folder in all_entries_in(arb_sud_folder):
#     if region_folder not in regions_parsed:
#         for year_folder in all_entries_in(region_folder):
#             for doc_xml_gz_path in all_entries_in(year_folder):
#                 doc_xml_gz = gzip.open(doc_xml_gz_path)
#                 s = BeautifulSoup(doc_xml_gz.read())
#                 doc_xml_gz.close()
#                 doc_tags = [
#                             text), s.region, s.court, s.vidpr, 
#                             s.etapd, s.category, s.date, s.vid_dokumenta
#                         ]
#                 if doc_tags[0] is None:
#                     doc_tags[0] = s.case_number
#                 doc_info = [tag.text if tag else None for tag in doc_tags]
#                 solutions_info.append(doc_info)
#                 pbar.update(1)
#                 progress_counter += 1
#         regions_parsed.add(region_folder)

#         with open('solutions_info.json', 'w') as f:
#             json.dump(solutions_info, f)

#         with open('progress_counter.json', 'w') as f:
#             json.dump([progress_counter], f)

#         with open('regions_parsed.json', 'w') as f:
#             json.dump(list(regions_parsed), f)a

In [2]:
# solutions_info_df = pd.read_csv('data/t_solutions_info_df.csv', index_col=0, dtype='str')

Trying to map some category names

In [245]:
# # category_name_map = {}
# # not_regognized = set()
# for cat_in in solutions_info_df['category'].dropna().unique():
#     if cat_in in category_name_map or cat_in in not_regognized:
#         continue
#     print('ello')
#     distances = []
#     for _, cat_out_obj in categs_info_df.drop(categs_info_df.tail(1).index)[['сCode', 'Descr']].iterrows():
#         cat_out_idx = cat_out_obj['сCode']
#         cat_out = cat_out_obj['Descr']
#         d = Levenshtein.distance(cat_in, cat_out)
#         distances.append((cat_out_idx, cat_out, d))
#     distances = sorted(distances, key=lambda x: x[-1])
#     d_min = distances[0][-1]
#     if d_min < 45:
#         n = 3
#         candidate_distances = distances[:n]
#         print(f'Input category :: {cat_in}')
#         for i, (cat_out_idx, cat_out, d) in enumerate(candidate_distances, 1):
#             print(f'\t{i}. {cat_out_idx} {cat_out}. d :: {d}')
#         input_deсision = int(input('Input 0 for None: '))
#         if input_deсision in range(1, n+1):
#             decision_cat_out = candidate_distances[input_deсision-1][1]
#             category_name_map[cat_in] = decision_cat_out
#         else:
#             not_regognized.add(cat_in)
#         display.clear_output()
#     else:
#         not_regognized.add(cat_in)

In [258]:
with open('data/not_recognized.json', 'w') as f:
    json.dump(list(not_regognized), f, indent='\t', ensure_ascii=False)

---

## AutoKad -> Standart Form (Saving)

In [3]:
data = json.load(open(
    os.path.join(
#         os.path.expanduser('~'), 
        'kad.arbitr-parser/data.json',
    ), 'r'
))
reload(utils);

In [4]:
z = 0
for cat in data:
    z += len(data[cat])
z

19394

In [5]:
utils.mkdir('raw_data/autoKad_data')

root_dir = os.path.join(
    os.curdir,
    'raw_data',
    'autoKad_data'
)

for category in data:
    for document in data[category]:
        date, court, text = document
        # FINDING CASENUMBER FROM TEXT =====
        casenumber_raw, cn_is_extracted = utils.extract_casenumber(doc=text)
        casenumber, cn_is_processed = utils.process_raw_casenumber(casenumber_raw)
        # ===================================
        year = date.split('.')[-1]
        court_dir = os.path.join(root_dir, court)
        year_dir = os.path.join(court_dir, year)
        utils.mkdir(court_dir)
        utils.mkdir(year_dir)
        file_path = os.path.join(year_dir,
            hashlib.sha1(' '.join(document + [category]).encode('utf-8')).hexdigest() + '.html.gz'
        )
        doc_html_string = utils.make_xml_string(*document, casenumber, category)
        doc_html_bytes = bytes(doc_html_string, 'utf-8')
        f_out = gzip.open(file_path, 'wb')
        f_out.write(doc_html_bytes)
        f_out.close()

## Updating catID's in cases df in agreement with taxonomy_df
### Building cases_df for autoKad data

In [13]:
cases_df_old = pd.read_csv('temp_data/cases_info_old.csv', index_col=0)
old_categs_df = pd.read_csv('temp_data/categs_info_old.csv', index_col=0)
taxonomy_df = pd.read_csv('data/taxonomies/taxonomy_df.csv')
oldID2newID = json.load(open('temp_data/oldID2newID_dict.json', 'r'), 
                        object_hook = (lambda d: {int(k):int(v) for k, v in d.items()})
)

/home/whoisjiji/.local/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [61]:
cases_df_new = cases_df_old.copy()
cases_df_new['CategoryID'] = cases_df_old['CategoryID'].map(
                                lambda x: oldID2newID.get(x, -1)
)
cases_df_new.to_csv('data/cases_info.csv', index=False)

In [54]:
kada_data = []
read_dir = 'raw_data/autoKad_data/'
for doc in utils.yield_document(read_dir):
    s = BeautifulSoup(doc, 'html.parser')
    doc_tags = [
        s.court, s.casenumber, s.category, s.date,
        ]
    doc_info = [tag.text if tag else None for tag in doc_tags]
    court, casenumber, category, date = doc_info
    
    # we care only about docs with casenumber and category from taxonomy_df
    if (casenumber is not None) and (category is not None) and (not utils.date_is_anomal(date)):
        date = utils.change_date_format(date)
        cat_index, *cat_name_list = category.split('. ')
        cat_name = '. '.join(cat_name_list)
        cCode_mask = taxonomy_df['cCode'] == cat_index
        cat_is_in_taxonomy = cCode_mask.sum() == 1
        if cat_is_in_taxonomy:
            ID = taxonomy_df[cCode_mask]['ID'].values[0]
            kada_data.append([casenumber, ID, -1, date, date])
kada_cases_df = pd.DataFrame(kada_data, columns=cases_df_new.columns)
kada_cases_df.to_csv('temp_data/kada_cases_info.csv', index=False)

## Making 2 stratified samples (large and smalll) of Suvorov Data collection

In [102]:
regions = set(json.load(open('data/regions.json', 'r')))
courts = set(json.load(open('data/courts.json', 'r')))

In [1]:
N_LARGE = 10**6
N_SMALL = 10**5

In [8]:
solutions_info_df = pd.read_csv('data/t_solutions_info_df.csv', index_col=0, dtype='str')

In [124]:
region_mask = solutions_info_df['region'].isin(regions)
court_mask =  solutions_info_df['court'].isin(courts)
first_inst_mask = solutions_info_df['etapd'] == 'Первая инстанция'

labeled_mask = solutions_info_df['casenumber'].isin(case_to_category_dict)

overall_mask = (
    labeled_mask &
    region_mask &
    court_mask &
    first_inst_mask
)

In [202]:
df = solutions_info_df[overall_mask]
df = df.assign(
    label = lambda x: x['casenumber'].apply(lambda x: case_to_category_dict[x]).values, 
)
df = df.assign(
    strata = lambda x: x['court'] + '.' +  x['region'] + '.' + x['label'].astype(str)
)

In [267]:
large_sample_df = (
    df.groupby('strata', group_keys=False)
        .apply(lambda x: x.sample(frac=N_LARGE/len(df), random_state=1))
)

small_sample_df = (
    df.groupby('strata', group_keys=False)
        .apply(lambda x: x.sample(frac=N_SMALL/len(df), random_state=1))
)

large_sample_df.to_csv('data/t_solutions_info_df_large_sample.csv') 
small_sample_df.to_csv('data/t_solutions_info_df_small_sample.csv')

large_case_numbers = set(large_sample_df['casenumber'].unique())
small_case_numbers = set(small_sample_df['casenumber'].unique())

In [248]:
# mkdir('data/cases_large')
# mkdir('data/cases_small')

In [287]:
cases_large_folder = os.path.join(os.curdir, 'data', 'cases_large')
cases_small_folder = os.path.join(os.curdir, 'data', 'cases_small')

In [284]:
def copy_file_to_case_folder(source, cases_folder):
    residual, source_name = os.path.split(source)
    residual, year = os.path.split(residual)
    residual, region = os.path.split(residual)

    region_folder = os.path.join(cases_folder, region)
    mkdir(region_folder)
    year_folder = os.path.join(region_folder, year)
    mkdir(year_folder)

    destination = os.path.join(year_folder, source_name)

    shutil.copyfile(source, destination)

In [293]:
progress_counter = 0

In [294]:
pbar = tqdm(total=n_files)
pbar.update(progress_counter)
cases_large_folder = os.path.join(os.curdir, 'data', 'cases_large')
cases_small_folder = os.path.join(os.curdir, 'data', 'cases_small')
for region_folder in all_entries_in(arb_sud_folder):
    for year_folder in all_entries_in(region_folder):
        for doc_xml_gz_path in all_entries_in(year_folder):
            doc_xml_gz = gzip.open(doc_xml_gz_path)
            s = BeautifulSoup(doc_xml_gz.read())
            doc_xml_gz.close()
            casenumber = s.casenumber
            if casenumber is None:
                casenumber = s.case_number
            if casenumber:
                casenumber = casenumber.text

            if casenumber in large_case_numbers:
                copy_file_to_case_folder(doc_xml_gz_path, cases_large_folder)
            if casenumber in small_case_numbers:
                copy_file_to_case_folder(doc_xml_gz_path, cases_small_folder)
                
            pbar.update(1)
            progress_counter += 1

  0%|          | 0/14810696 [00:00<?, ?it/s]

In [62]:
progress_counter=0
pbar = tqdm(total=n_files)
for region_folder in all_entries_in(arb_sud_folder):
    for year_folder in all_entries_in(region_folder):
        for doc_xml_gz_path in all_entries_in(year_folder):
            doc_xml_gz = gzip.open(doc_xml_gz_path)
            doc = doc_xml_gz.read().decode('utf8')
            s = BeautifulSoup(doc, 'html.parser')
            
            pbar.update(1)
            progress_counter += 1

---

## CREATING CATEGS MAP DF

In [66]:
#### CATEGS INFO ##########################################
categs_info = pd.read_csv('temp_data/categs_info_old.csv')
cCode2descr = dict(zip(
                categs_info['cCode'].str.strip(), 
                categs_info['Descr'].str.strip()
))
cCode2descr.pop('Нет', None)
descr2cCode = {val:key for key, val in cCode2descr.items()}
############################################################

### CREATING categs mapping df #############################
rcg_dict = json.load(open('temp_data/category_name_map.json'))
not_rcg_list = json.load(open('temp_data/not_recognized.json'))
not_rcg_dict = {name:'' for name in not_rcg_list}

categs_name_df = pd.DataFrame(
    list(rcg_dict.items()) + 
    list(not_rcg_dict.items())
)
categs_name_df.rename(columns={0:'torent', 1:'kadarbitr'}, inplace=True)
categs_name_df['kadarbitr'] = (
    categs_name_df['kadarbitr'].map(lambda descr: descr2cCode.get(descr, '')) + 
    '. ' 
    + categs_name_df['kadarbitr']
)  # adding index to names
categs_name_df['kadarbitr'][categs_name_df['kadarbitr'] == '. '] = ''
categs_name_df.to_csv('data/categs_map_df.csv', index=False)

## PREPROCESSING AUTOKAD DATA v1

In [18]:
reload(utils)
read_dir = 'raw_data/autoKad_data/'
save_dir = 'data/autoKad_data_v1/'
utils.mkdir(save_dir)
z = []
for doc_path, doc in utils.yield_document2(read_dir):
    court, casenumber, category, date, text  = utils.process_xml_string(doc)
    z.append(p_doc_path)
    p_text_list = utils.preprocess_text(text, lemmatize=True)
    p_text = ' '.join(p_text_list)
    
    #### extracting YEAR and REG and NAME from PATH ########
    year_folder, name = os.path.split(doc_path)
    region_folder, year = os.path.split(year_folder)
    _, region = os.path.split(region_folder)
    ###############################################
    p_region_folder = os.path.join(save_dir, region)
    utils.mkdir(p_region_folder)
    p_year_folder = os.path.join(p_region_folder, year)
    utils.mkdir(p_year_folder)
    p_doc_path = os.path.join(p_year_folder, name)
    ###############################################
    doc_html_string = utils.make_xml_string(date, court, p_text, casenumber, category)
    doc_html_bytes = bytes(doc_html_string, 'utf-8')
    f_out = gzip.open(p_doc_path, 'wb')
    f_out.write(doc_html_bytes)
    f_out.close()  